In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

df = pd.read_csv("landmarks_dataset.csv")

# Separar datos y etiquetas
X = df.drop(columns=["label"])
y = df["label"]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar modelo Random Forest
#rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf = RandomForestClassifier(
    n_estimators=300,  # Más árboles
    max_depth=20,  # Limitar profundidad
    max_features='sqrt',  # Reducir número de características evaluadas por nodo
    random_state=42
)

rf.fit(X_train, y_train)

# Evaluar modelo
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ Precisión del modelo: {accuracy:.2%}")

joblib.dump(rf, "modelo_random_forest.pkl")
print("💾 Modelo guardado como modelo_random_forest.pkl")

✅ Precisión del modelo: 66.82%
💾 Modelo guardado como modelo_random_forest.pkl


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import os

# Cargar modelo Random Forest
model = joblib.load("modelo_random_forest.pkl")

#Obtener los nombres de las poses (las carpetas del dataset)
dataset_folder = "dataset"
pose_names = os.listdir(dataset_folder)

#Inicializar MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Cargar imagen a predecir
img_path = "fotos/fotoDetectar.jpg"
image = cv2.imread(img_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Procesar imagen y extraer landmarks
results = pose.process(image_rgb)

if results.pose_landmarks:
    landmarks = []
    for lm in results.pose_landmarks.landmark:
        landmarks.extend([lm.x, lm.y, lm.z])

    #Predecir clase
    prediction = model.predict([landmarks])[0]  # modelo espera array de arrays

    print(f"📸 Pose predicha: {prediction}")
else:
    print("⚠️ No se detectaron landmarks en la imagen.")

pose.close()

📸 Pose predicha: ardha chandrasana


C:\Users\USUARIO\miniconda3\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
